### Open namepsace and get exetnsions for multichannel volumetric data

In [24]:
import os
from pynwb import load_namespaces, get_class
from pynwb.file import MultiContainerInterface, NWBContainer
import skimage.io as skio
from collections.abc import Iterable
import numpy as np
from pynwb import register_class
from hdmf.utils import docval, get_docval, popargs
from pynwb.ophys import ImageSeries 
from pynwb.core import NWBDataInterface
from hdmf.common import DynamicTable
from hdmf.utils import docval, popargs, get_docval, get_data_shape, popargs_to_dict
from pynwb.file import Device
import pandas as pd
import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
from pynwb.ophys import OnePhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries
from datetime import datetime
from dateutil import tz
import pandas as pd
import scipy.io as sio
from datetime import datetime, timedelta
import tifffile

# Set path of the namespace.yaml file to the expected install location
MultiChannelVol_specpath = os.path.join(
    os.getcwd(),
    'spec',
    'ndx-multichannel-volume.namespace.yaml'
)
# Load the namespace
load_namespaces(MultiChannelVol_specpath)

# TODO: import your classes here or define your class using get_class to make
# them accessible at the package level
#MultiChannelVolume = get_class('MultiChannelVolume', 'ndx-multichannel-volume')
#ImagingVolume = get_class('ImagingVolume', 'ndx-multichannel-volume')
CElegansSubject = get_class('CElegansSubject', 'ndx-multichannel-volume')
OpticalChannelReferences = get_class('OpticalChannelReferences', 'ndx-multichannel-volume')
OpticalChannelPlus = get_class('OpticalChannelPlus', 'ndx-multichannel-volume')
#VolumeSegmentation = get_class('VolumeSegmentation', 'ndx-multichannel-volume')

In [25]:
datapath = os.path.join('/Users', 'danielsprague', 'FOCO_lab', 'data')

### Define additional functionality for classes

In [26]:
'''
@register_class('OpticalChannelPlus', 'ndx-multichannel-volume')
class OpticalChannelPlus(OpticalChannel):
    """An optical channel used to record from an imaging plane."""

    __nwbfields__ = ('excitation_lambda',
                     'emission_lambda',
                     'emission_range',
                     'excitation_range')

    @docval(*get_docval(OpticalChannel.__init__),
            {'name': 'excitation_lambda', 'type': float, 'doc': 'Excitation wavelength for channel, in nm'}, # required
            {'name': 'emission_range', 'type': ('data', 'array_data'), 'doc': 'boundaries of emission wavelength for channel, in nm'},
            {'name': 'excitation_range', 'type': ('data', 'array_data'), 'doc': 'boundaries of excitation wavelength for channel, in nm'})  # required
    def __init__(self, **kwargs):
        excitation_lambda, emission_range, excitation_range, emission_lambda = popargs("excitation_lambda", "emission_range", "excitation_range", kwargs)
        super().__init__(**kwargs)
        self.emission_range = emission_range
        self.excitation_range = excitation_range
        self.excitation_lambda = excitation_lambda
'''

'\n@register_class(\'OpticalChannelPlus\', \'ndx-multichannel-volume\')\nclass OpticalChannelPlus(OpticalChannel):\n    """An optical channel used to record from an imaging plane."""\n\n    __nwbfields__ = (\'excitation_lambda\',\n                     \'emission_lambda\',\n                     \'emission_range\',\n                     \'excitation_range\')\n\n    @docval(*get_docval(OpticalChannel.__init__),\n            {\'name\': \'excitation_lambda\', \'type\': float, \'doc\': \'Excitation wavelength for channel, in nm\'}, # required\n            {\'name\': \'emission_range\', \'type\': (\'data\', \'array_data\'), \'doc\': \'boundaries of emission wavelength for channel, in nm\'},\n            {\'name\': \'excitation_range\', \'type\': (\'data\', \'array_data\'), \'doc\': \'boundaries of excitation wavelength for channel, in nm\'})  # required\n    def __init__(self, **kwargs):\n        excitation_lambda, emission_range, excitation_range, emission_lambda = popargs("excitation_lambda

In [27]:
@register_class('ImagingVolume', 'ndx-multichannel-volume')
class ImagingVolume(NWBDataInterface):
    """An imaging plane and its metadata."""

    __nwbfields__ = ({'name': 'optical_channel_plus', 'child': True},
                     'Order_optical_channels',
                     'description',
                     'device',
                     'location',
                     'conversion',
                     'origin_coords',
                     'origin_coords_units',
                     'grid_spacing',
                     'grid_spacing_units',
                     'reference_frame',
                     )

    @docval(*get_docval(NWBDataInterface.__init__, 'name'),  # required
            {'name': 'optical_channel_plus', 'type': ('data', 'array_data', OpticalChannelPlus),  # required
             'doc': 'One of possibly many groups storing channel-specific data.'},
            {'name': 'Order_optical_channels', 'type':OpticalChannelReferences, 'doc':'Order of the optical channels in the data'},
            {'name': 'description', 'type': str, 'doc': 'Description of this ImagingVolume.'},  # required
            {'name': 'device', 'type': Device, 'doc': 'the device that was used to record'},  # required
            {'name': 'location', 'type': str, 'doc': 'Location of image plane.'},  # required
            {'name': 'reference_frame', 'type': str,
             'doc': 'Describes position and reference frame of manifold based on position of first element '
                    'in manifold.',
             'default': None},
            {'name': 'origin_coords', 'type': 'array_data',
             'doc': 'Physical location of the first element of the imaging plane (0, 0) for 2-D data or (0, 0, 0) for '
                    '3-D data. See also reference_frame for what the physical location is relative to (e.g., bregma).',
             'default': None},
            {'name': 'origin_coords_unit', 'type': str,
             'doc': "Measurement units for origin_coords. The default value is 'meters'.",
             'default': 'meters'},
            {'name': 'grid_spacing', 'type': 'array_data',
             'doc': "Space between pixels in (x, y) or voxels in (x, y, z) directions, in the specified unit. Assumes "
                    "imaging plane is a regular grid. See also reference_frame to interpret the grid.",
             'default': None},
            {'name': 'grid_spacing_unit', 'type': str,
             'doc': "Measurement units for grid_spacing. The default value is 'meters'.",
             'default': 'meters'})
    def __init__(self, **kwargs):
        keys_to_set = ('optical_channel_plus',
                       'Order_optical_channels',
                       'description',
                       'device',
                       'location',
                       'reference_frame',
                       'origin_coords',
                       'origin_coords_unit',
                       'grid_spacing',
                       'grid_spacing_unit')
        args_to_set = popargs_to_dict(keys_to_set, kwargs)
        super().__init__(**kwargs)

        if not isinstance(args_to_set['optical_channel_plus'], list):
            args_to_set['optical_channel_plus'] = [args_to_set['optical_channel_plus']]

        for key, val in args_to_set.items():
            setattr(self, key, val)

In [28]:
@register_class('VolumeSegmentation', 'ndx-multichannel-volume')
class VolumeSegmentation(DynamicTable):
    """
    Stores pixels in an image that represent different regions of interest (ROIs)
    or masks. All segmentation for a given imaging volume is stored together, with
    storage for multiple imaging planes (masks) supported. Each ROI is stored in its
    own subgroup, with the ROI group containing both a 3D mask and a list of pixels
    that make up this mask. Segments can also be used for masking neuropil. If segmentation
    is allowed to change with time, a new imaging plane (or module) is required and
    ROI names should remain consistent between them.
    """

    __fields__ = ('imaging_volume','name')

    __columns__ = (
        {'name': 'image_mask', 'description': 'Image masks for each ROI'},
        {'name': 'voxel_mask', 'description': 'Voxel masks for each ROI', 'index': True},
        {'name': 'color_voxel_mask', 'description': 'Color voxel masks for each ROI', 'index':True}
    )

    @docval({'name': 'description', 'type': str,  # required
             'doc': 'Description of image plane, recording wavelength, depth, etc.'},
            {'name': 'imaging_volume', 'type': ImagingVolume,  # required
             'doc': 'the ImagingVolume this ROI applies to'},
            {'name': 'name', 'type': str, 'doc': 'name of VolumeSegmentation.', 'default': None},
            *get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        imaging_volume = popargs('imaging_volume', kwargs)
        if kwargs['name'] is None:
            kwargs['name'] = imaging_volume.name
        super().__init__(**kwargs)
        self.imaging_volume = imaging_volume

    @docval({'name': 'voxel_mask', 'type': 'array_data', 'default': None,
             'doc': 'voxel mask for 3D ROIs: [(x1, y1, z1, weight1, ID), (x2, y2, z2, weight2, ID), ...]',
             'shape': (None, 5)},
             {'name': 'color_voxel_mask', 'type': 'array_data', 'default': None,
             'doc': 'voxel mask for 3D ROIs with color information',
             'shape': (None, 9)},
            {'name': 'image_mask', 'type': 'array_data', 'default': None,
             'doc': 'image with the same size of image where positive values mark this ROI',
             'shape': [[None]*3]},
            {'name': 'id', 'type': int, 'doc': 'the ID for the ROI', 'default': None},
            allow_extra=True)
    def add_roi(self, **kwargs):
        """Add a Region Of Interest (ROI) data to this"""
        voxel_mask, color_voxel_mask, image_mask = popargs('voxel_mask', 'color_voxel_mask', 'image_mask', kwargs)
        if image_mask is None and voxel_mask is None and color_voxel_mask is None:
            raise ValueError("Must provide 'image_mask' and/or 'voxel_mask' and/or 'color_voxel_mask'")
        rkwargs = dict(kwargs)
        if image_mask is not None:
            rkwargs['image_mask'] = image_mask
        if voxel_mask is not None:
            rkwargs['voxel_mask'] = voxel_mask
        if color_voxel_mask is not None:
            rkwargs['color_voxel_mask'] = color_voxel_mask
        return super().add_row(**rkwargs)

    @staticmethod
    def voxel_to_image(voxel_mask):
        """Converts a #D pixel_mask of a ROI into an image_mask."""
        image_matrix = np.zeros(np.shape(voxel_mask))
        npmask = np.asarray(voxel_mask)
        x_coords = npmask[:, 0].astype(np.int32)
        y_coords = npmask[:, 1].astype(np.int32)
        z_coords = npmask[:, 2].astype(np.int32)
        weights = npmask[:, -1]
        image_matrix[y_coords, x_coords, z_coords] = weights
        return image_matrix

    @staticmethod
    def image_to_pixel(image_mask):
        """Converts an image_mask of a ROI into a pixel_mask"""
        voxel_mask = []
        it = np.nditer(image_mask, flags=['multi_index'])
        while not it.finished:
            weight = it[0][()]
            if weight > 0:
                x = it.multi_index[0]
                y = it.multi_index[1]
                z = it.multi_index[2]
                voxel_mask.append([x, y, z, weight])
            it.iternext()
        return voxel_mask

    @docval({'name': 'description', 'type': str, 'doc': 'a brief description of what the region is'},
            {'name': 'region', 'type': (slice, list, tuple), 'doc': 'the indices of the table', 'default': slice(None)},
            {'name': 'name', 'type': str, 'doc': 'the name of the ROITableRegion', 'default': 'rois'})
    def create_roi_table_region(self, **kwargs):
        return self.create_region(**kwargs)
    


In [29]:
@register_class('MultiChannelVolume', 'ndx-multichannel-volume')
class MultiChannelVolume(NWBDataInterface):
    """An imaging plane and its metadata."""

    __nwbfields__ = ('resolution',
                     'description',
                     'RGBW_channels',
                     'data',
                     'imaging_volume'
                     )

    @docval(*get_docval(NWBDataInterface.__init__, 'name'),  # required
            {'name': 'resolution', 'type': 'array_data', 'doc':'pixel resolution of the image', 'shape':[None]},
            {'name': 'imaging_volume', 'type': ImagingVolume, 'doc': 'the Imaging Volume the data was generated from'},
            {'name': 'description', 'type': str, 'doc':'description of image'},
            {'name': 'RGBW_channels', 'doc': 'which channels in image map to RGBW', 'type': 'array_data', 'shape':[None]},
            {'name': 'data', 'doc': 'Volumetric multichannel data', 'type': 'array_data', 'shape':[None]*4},
    )
    
    def __init__(self, **kwargs):
        keys_to_set = ('resolution',
                       'description',
                       'RGBW_channels',
                       'data',
                       'imaging_volume'
                       )
        args_to_set = popargs_to_dict(keys_to_set, kwargs)
        super().__init__(**kwargs)

        for key, val in args_to_set.items():
            setattr(self, key, val)

### Creating NWB file for NeuroPAL data

In [30]:
def gen_file(description, identifier, start_date_time, lab, institution, pubs):

    nwbfile = NWBFile(
        session_description = description,
        identifier = identifier,
        session_start_time = start_date_time,
        lab = lab,
        institution = institution,
        related_publications = pubs
    )

    return nwbfile

In [31]:
def create_im_vol(device, channels, location="head", grid_spacing=[0.3208, 0.3208, 0.75], grid_spacing_unit ="micrometers", origin_coords=[0,0,0], origin_coords_unit="micrometers", reference_frame="Worm head, left=anterior, bottom=ventral"):
    
    # channels should be ordered list of tuples (name, description)

    OptChannels = []
    OptChanRefData = []
    for name, wave in channels:
        excite = float(wave.split('-')[0])
        emiss_mid = float(wave.split('-')[1])
        emiss_range = float(wave.split('-')[2][:-1])
        OptChan = OpticalChannelPlus(
            name = name,
            description = wave,
            excitation_lambda = excite,
            excitation_range = [excite, excite],
            emission_range = [emiss_mid-emiss_range/2, emiss_mid+emiss_range/2],
            emission_lambda = emiss_mid
        )

        OptChannels.append(OptChan)
        OptChanRefData.append(wave)

    OpticalChannelRefs = OpticalChannelReferences(
        name = 'OpticalChannelRefs',
        data = OptChanRefData
    )

    imaging_vol = ImagingVolume(
        name= 'ImagingVolume',
        optical_channel_plus = OptChannels,
        Order_optical_channels = OpticalChannelRefs,
        description = 'NeuroPAL image of C elegan brain',
        device = device,
        location = location,
        grid_spacing = grid_spacing,
        grid_spacing_unit = grid_spacing_unit,
        origin_coords = origin_coords,
        origin_coords_unit = origin_coords_unit,
        reference_frame = reference_frame
    )

    return imaging_vol, OpticalChannelRefs, OptChannels

In [32]:
def create_vol_seg(imaging_vol, blobs):

    vs = VolumeSegmentation(
        name = 'VolumeSegmentation',
        description = 'Neuron centers for multichannel volumetric image',
        imaging_volume = imaging_vol
    )

    csv = pd.read_csv(blobs)

    voxel_mask = []

    for i, row in csv.iterrows():
        x = row['X']
        y = row['Y']
        z = row['Z']
        ID = row['ID']

        voxel_mask.append([np.uint(x),np.uint(y),np.uint(z),1,str(ID)])

    vs.add_roi(voxel_mask=voxel_mask)

    return vs

In [33]:
def create_image(data, name, description, imaging_volume, resolution=[0.3208, 0.3208, 0.75], RGBW_channels=[0,1,2,3]):

    image = MultiChannelVolume(
        name = name,
        resolution = resolution,
        description = description,
        RGBW_channels = RGBW_channels,
        data = data,
        imaging_volume = imaging_volume
    )

    return image

In [34]:
'''
Create NWB file from tif file of raw image, neuroPAL software created mat file and csv files of blob locations
'''

def create_file_FOCO(folder, reference_frame):


    worm = folder.split('/')[1]

    path = datapath+'/'+folder

    for file in os.listdir(path):
        if file[-4:] =='.tif':
            imfile = path + '/'+file

        elif file[-4:] == '.mat' and file[-6:]!= 'ID.mat':
            matfile = path + '/'+file

        elif file == 'blobs.csv':
            blobs = path +'/'+file

    data = np.transpose(skio.imread(imfile), (1,0,2,3))
    #data = data.astype('uint16')
    mat = sio.loadmat(matfile)

    scale = np.asarray(mat['info']['scale'][0][0]).flatten()
    prefs = np.asarray(mat['prefs']['RGBW'][0][0]).flatten()-1 #subtract 1 to adjust for matlab indexing from 1
    
    dt = worm.split('-')
    session_start = datetime(int(dt[0]),int(dt[1]),int(dt[2]), tzinfo=tz.gettz("US/Pacific"))
    
    nwbfile = gen_file('Worm head', worm, session_start, 'Kato lab', 'UCSF', "")

    nwbfile.subject = CElegansSubject(
    subject_id = worm,
    #age = "T2H30M",
    #growth_stage_time = pd.Timedelta(hours=2, minutes=30).isoformat(),
    date_of_birth = session_start, #currently just using the session start time to bypass the requirement for date of birth
    growth_stage = 'YA',
    cultivation_temp = 20.,
    description = dt[3]+'-'+dt[4],
    species  = "Caenorhabditis elegans",
    sex = "O", #currently just using O for other until support added for other gender specifications
    strain = "OH16230"
    )

    device = nwbfile.create_device(
    name = "Microscope",
    description = "One-photon microscope Weill",
    manufacturer = "Leica"
    )

    channels = [("mNeptune 2.5", "561-700-75m"), ("Tag RFP-T", "561-605-70m"), ("CyOFP1", "488-605-70m"), ("GFP-GCaMP", "488-525-50m"), ("mTagBFP2", "405-460-50m"), ("mNeptune 2.5 - high excite", "639-700-75m")]
    
    ImagingVol, OptChannelRefs, OpticalChannelPlus = create_im_vol(device, channels, location= "head", grid_spacing= scale, reference_frame=reference_frame)

    vs = create_vol_seg(ImagingVol, blobs)

    image= create_image(data, 'NeuroPALImageRaw', worm, ImagingVol, resolution=scale, RGBW_channels=[0,2,4,1])

    nwbfile.add_acquisition(image)

    neuroPAL_module = nwbfile.create_processing_module(
        name = 'NeuroPAL',
        description = 'neuroPAL image data and metadata',
    )    

    processed_im_module = nwbfile.create_processing_module(
        name = 'ProcessedImage',
        description = 'Pre-processed image. Currently median filtered and histogram matched to original neuroPAL images.'
    )

    proc_imfile = datapath + '/NP_FOCO_hist_med/'+worm+'/hist_med_image.tif'

    proc_data = np.transpose(skio.imread(proc_imfile), (0,3,1,2))
    #proc_data = proc_data.astype('uint16')

    proc_image = create_image(proc_data, 'Hist_match_med_filt', worm, ImagingVol, resolution=scale, RGBW_channels=[0,1,2,3])

    neuroPAL_module.add(vs)
    neuroPAL_module.add(ImagingVol)
    neuroPAL_module.add(OptChannelRefs)
    neuroPAL_module.add(OpticalChannelPlus)

    processed_im_module.add(proc_image)

    io = NWBHDF5IO(datapath+'/nwb/'+worm+'.nwb', mode='w')
    io.write(nwbfile)
    io.close()


    

In [35]:
create_file_FOCO('NP_FOCO_cropped/2021-12-03-w00-NP1', reference_frame = 'origin = anterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-01-22-w04-NP1', reference_frame = 'origin = posterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-02-11-w03-NP1', reference_frame = 'origin = posterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-02-12-w00-NP1', reference_frame = 'origin = posterior, ventral, right')
create_file_FOCO('NP_FOCO_cropped/2022-02-12-w01-NP1', reference_frame = 'origin = posterior, ventral, right')
create_file_FOCO('NP_FOCO_cropped/2022-02-22-w04-NP1', reference_frame = 'origin = anterior, ventral left; slightly rotated')
create_file_FOCO('NP_FOCO_cropped/2022-03-05-w00-NP1', reference_frame = 'origin = posterior, ventral, right')
create_file_FOCO('NP_FOCO_cropped/2022-04-01-w00-NP1', reference_frame = 'origin = anterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-04-26-w00-NP1', reference_frame = 'origin = posterior, dorsal, left; slightly rotated')
create_file_FOCO('NP_FOCO_cropped/2022-04-26-w01-NP1', reference_frame = 'origin = posterior, ventral, right')

/Users/danielsprague/anaconda3/envs/nwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'ImagingVolume/origin_coords': Value with data type int64 is being converted to data type float64 (min specification: float32).
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/danielsprague/anaconda3/envs/nwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'ImagingVolume/origin_coords': Value with data type int64 is being converted to data type float64 (min specification: float32).
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/danielsprague/anaconda3/envs/nwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'ImagingVolume/origin_coords': Value with data type int64 is being converted to data type float64 (min specification: float32).
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/danielsprague/anaconda3/envs/nwb/lib/python3.8/s

: 

### Read NWB files and recreate relevant files

In [13]:
with NWBHDF5IO(datapath+"/nwb/2022-02-12-w01-NP1.nwb", mode='r', load_namespaces=True) as io:
    read_nwbfile = io.read()
    subject = read_nwbfile.subject
    image = read_nwbfile.acquisition['NeuroPALImageRaw'].data[:]
    resolution = read_nwbfile.acquisition['NeuroPALImageRaw'].resolution[:]
    channels = read_nwbfile.acquisition['NeuroPALImageRaw'].RGBW_channels[:]
    seg = read_nwbfile.processing['NeuroPAL']['VolumeSegmentation'].voxel_mask[:]
    im_vol = read_nwbfile.processing['NeuroPAL']['ImagingVolume']
    optchans = read_nwbfile.processing['NeuroPAL']['ImagingVolume'].optical_channel_plus[:]
    grid_spacing = im_vol.grid_spacing[:]
    chan_refs = read_nwbfile.processing['NeuroPAL']['OpticalChannelRefs'].data[:]
    proc_image = read_nwbfile.processing['ProcessedImage']['Hist_match_med_filt'].data[:]



In [14]:
'''
Recreate blobs.csv from vol_seg
'''

blobs = pd.DataFrame.from_records(seg, columns=['X', 'Y','Z','weight', 'ID'])
blobs = blobs.drop(['weight'], axis=1)
blobs = blobs.replace('nan', '', regex=True)
blobs.to_csv(datapath+'/nwb/test.csv')


In [15]:
'''
Recreate original image from acquisition raw image
'''
print(image.shape)

#im = np.transpose(image,(2,3,1,0)) #have to flip some of the axes to align with how FIJI reads tiff images
#image = image.astype('uint16')
#print(im.dtype)
#print(im.shape)

tifffile.imwrite(datapath + '/nwb/test.tif', image, imagej = True)

(45, 6, 240, 1000)


In [16]:
'''
Recreate histogram matched image from processed data
'''

print(proc_image.shape)

#im = np.transpose(proc_image, (3,0,2,1)) #have to flip some of the axes to align with how FIJI reads tiff images
#proc_image = proc_image.astype('uint16')
print(proc_image.dtype)

tifffile.imwrite(datapath + '/nwb/test_proc.tif', proc_image, imagej = True)

(45, 4, 240, 1000)
uint16


In [28]:
print(resolution)
print(channels)

[0.3216 0.3216 0.75  ]
[0 2 4 1]


In [23]:
imfile = datapath + '/NP_FOCO_cropped/2022-02-12-w01-NP1/neuropal_1_MMStack_Pos0.ome.tif'
proc_imfile = datapath + '/NP_FOCO_hist_med/2022-02-12-w01-NP1/hist_med_image.tif'
data = skio.imread(imfile)
proc_data = skio.imread(proc_imfile)

print(data.shape)
print(proc_data.shape)


(6, 45, 240, 1000)
(45, 240, 1000, 4)


In [23]:
from dandi.metadata import get_metadata

meta = get_metadata('/Users/danielsprague/FOCO_lab/data/000472/../nwb/2022-02-12-w01-NP1.nwb')

print(meta)

{'nwb_version': '2.6.0', 'experiment_description': None, 'experimenter': None, 'identifier': '2022-02-12-w01-NP1', 'institution': 'UCSF', 'keywords': None, 'lab': 'Kato lab', 'related_publications': ('',), 'session_description': 'Worm head', 'session_id': None, 'session_start_time': datetime.datetime(2022, 2, 12, 0, 0, tzinfo=tzoffset(None, -28800)), 'age': None, 'date_of_birth': datetime.datetime(2022, 2, 12, 0, 0, tzinfo=tzoffset(None, -28800)), 'genotype': None, 'sex': 'O', 'species': 'Caenorhabditis elegans', 'subject_id': '2022-02-12-w01-NP1', 'number_of_electrodes': 0, 'number_of_units': 0, 'external_file_objects': [], 'nd_types': ['CElegansSubject', 'Device (5)', 'ImagingVolume (4)', 'MultiChannelVolume (2)', 'OpticalChannelPlus (24)', 'OpticalChannelReferences (5)', 'ProcessingModule (2)', 'VolumeSegmentation']}


In [ ]:


from dandi.organize import filter_invalid_metadata_rows(metadata)